In [1]:
import os

In [2]:
%pwd

'/home/suganya/Documents/Text-Summarization-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/suganya/Documents/Text-Summarization-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        print("DEBUG keys:", self.config.keys())
        print("DEBUG params keys:", self.params.keys())  # Add this to see what's available
        print("DEBUG eval_steps value:", self.params.get('eval_steps', 'NOT FOUND'))  # Add this
    
        # This CALLS the create_directories function (defined elsewhere)
        create_directories([self.config.artifacts_root])

    def  get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            eval_strategy= params.eval_strategy,
            eval_steps= params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps
            
        )

        return model_trainer_config



In [9]:
import os
from textSummarizer.logging import logger
from datasets import load_dataset, load_from_disk
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments

/home/suganya/Documents/Text-Summarization-Project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collector = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # ✅ Convert to string and check if exists
        data_path = str(self.config.data_path)
        print(f"Looking for data at: {data_path}")
        print(f"Absolute path: {os.path.abspath(data_path)}")
        print(f"Exists: {os.path.exists(data_path)}")
        
        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Dataset not found at {data_path}")
        
        dataset_samsum_pt = load_from_disk(data_path)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        print(f"Original dataset size - Train: {len(dataset_samsum_pt['test'])}, Eval: {len(dataset_samsum_pt['validation'])}")
    
        train_subset = dataset_samsum_pt["test"].select(range(2))
        eval_subset = dataset_samsum_pt["validation"].select(range(2))
        
        print(f"Using subset - Train: {len(train_subset)}, Eval: {len(eval_subset)}")

        # trainer_args = TrainingArguments(
        #     output_dir = self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps= self.config.warm_steps, 
        #     per_device_train_batch_size= self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps, evaluation_strategy=self.config.evaluation_strategy, eval_steps= self.config.eval_steps,
        #     save_steps=1e6, gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )


        trainer_args = TrainingArguments(
        output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=5,
        per_device_train_batch_size=1, per_device_eval_batch_size=1,
        weight_decay=0.01, logging_steps=1,
        logging_first_step=True,
        eval_strategy='steps', eval_steps=500, save_steps=1e6,
        gradient_accumulation_steps=16,
        disable_tqdm=False,
        report_to="none",
        dataloader_pin_memory=False
        #fp16=True, # Enable mixed precision training
        # #report_to="none" # Disable wandb logging
        )
        

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          processing_class= tokenizer, data_collator=seq2seq_data_collector,
                          train_dataset = train_subset,
                          #eval_dataset= eval_subset
                          )
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-10-13 20:05:31,814: INFO: common: yaml file: /home/suganya/Documents/Text-Summarization-Project/config/config.yaml loaded successfully]
[2025-10-13 20:05:31,833: INFO: common: yaml file: /home/suganya/Documents/Text-Summarization-Project/params.yaml loaded successfully]
DEBUG keys: dict_keys(['artifacts_root', 'data_ingestion', 'data_validation', 'data_transformation', 'model_trainer', 'model_evalution'])
DEBUG params keys: dict_keys(['TrainingArguments'])
DEBUG eval_steps value: NOT FOUND
[2025-10-13 20:05:31,835: INFO: common: created directory: artifacts]
[2025-10-13 20:05:31,839: INFO: common: created directory: /home/suganya/Documents/Text-Summarization-Project/artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Looking for data at: /home/suganya/Documents/Text-Summarization-Project/artifacts/data_transformation/samsum_dataset
Absolute path: /home/suganya/Documents/Text-Summarization-Project/artifacts/data_transformation/samsum_dataset
Exists: True
Original dataset size - Train: 819, Eval: 818
Using subset - Train: 2, Eval: 2


/home/suganya/Documents/Text-Summarization-Project/venv/lib/python3.12/site-packages/transformers/training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
